<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Start session

In [2]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("ML oprerations").getOrCreate()

23/06/04 16:48:05 WARN Utils: Your hostname, Katos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.208.2.13 instead (on interface en0)
23/06/04 16:48:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/04 16:48:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [8]:
%%bash
curl -o searchterms.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  227k  100  227k    0     0   134k      0  0:00:01  0:00:01 --:--:--  134k


In [ ]:
# Load the csv into a spark dataframe

In [9]:
# Load the CSV file into a DataFrame
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [3]:
# Get the number of rows
num_rows = df.count()

# Get the number of columns
num_columns = len(df.columns)

# Print the number of rows and columns
print("Number of rows: ", num_rows)
print("Number of columns: ", num_columns)

NameError: name 'df' is not defined

In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [16]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [34]:
# Get the data types of all columns
column_types = df.dtypes

# Print the data type of the "searchterm" column
print("Data type of searchterm column:", [dtype for col, dtype in column_types if col == "searchterm"][0])

Data type of searchterm column: string


In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [30]:
df.filter(df['searchterm'] == "gaming laptop").count()

499

In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [38]:
from pyspark.sql.functions import desc

# Group the DataFrame by searchterm and count the occurrences
grouped_df = df.groupBy("searchterm").count()

# Sort the search terms by count in descending order
sorted_df = grouped_df.orderBy(desc("count"))

# Print the top 5 most frequently used search terms
sorted_df.show(5, truncate=False)

+-------------+-----+
|searchterm   |count|
+-------------+-----+
|mobile 6 inch|2312 |
|mobile 5g    |2301 |
|mobile latest|1327 |
|laptop       |935  |
|tablet wifi  |896  |
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [41]:
from pyspark.ml.regression import LinearRegressionModel

# Load the sales prediction model
model = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [50]:
from pyspark.ml.feature import VectorAssembler

# This function converts a scalar number into a dataframe that can be used by the model to predict.
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[0,0,year,0]]
    columns = ["day", "month", "year", "searchterm"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

